In [15]:
import pandas as pd
import json
import numpy as np
import time

with open('data.json') as f:
    data = json.load(f)

%matplotlib qt

# Reading to dataframe
# Creating additional columns
gen = ((k, v['assetA']['bid'], v['assetA']['ask'], v['assetB']['bid'], v['assetB']['ask']) for k, v in data.items())
data_list = [x for x in gen]
df = pd.DataFrame(data_list, columns=['timestamp', 'assetA_bid', 'assetA_ask', 'assetB_bid', 'assetB_ask'])
df['delta_t'] = df['timestamp'].astype('uint64').diff()
df['trade_freq'] = df.apply (lambda row: 1/row['delta_t'], axis=1)
df['trade_freq_avg'] = df['trade_freq'].rolling(window=15).mean()
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df.set_index('datetime')

In [41]:

# Lets plot the signals again
w = 7
asset = 'B'
ax = df['asset'+asset+'_ask'].plot(style='.-', color='tab:orange')
df['asset'+asset+'_ask'].rolling(w).mean().shift(int(-w/2)).plot(ax = ax, style=':', color='tab:orange')
ax = df['asset'+asset+'_bid'].plot(style='.-', color='tab:blue')
df['asset'+asset+'_bid'].rolling(w).mean().shift(int(-w/2)).plot(ax = ax, style=':', color='tab:blue')



In [42]:
# Simple market making 'on DataFrame' trading functions
def compute_trading_decision(row, spread_trade_margin = 0.2, rise_th = 0.05, fall_th = 0.05):
    # A simple function to decide on operation at moment t based on params and market status
    # rise_th - a threshold to detect market rising situation
    # fall_th - a threshold to detect market rising situation
    # spread_trade_margin - a margin from the ask/bid trends at which mm trade is considered profitable 
    operation = None
    if not row['gradient'] > rise_th: # if not rising
        if row['bid'] > row['trailing_ask'] - spread_trade_margin*row['spread']:
            operation = 'sell'
    if not row['gradient'] < fall_th: # if not falling
        if row['ask'] < row['trailing_ask'] + spread_trade_margin*row['spread']:
            operation = 'buy'

    return operation

def compute_trades(df, avg_w = 5, spread_trade_margin = 0.2, rise_th = 0.05, fall_th = 0.05):
    # Very simple market making starategy
    # if not rising
    # spread = abs(trailing ask - training bid)
    # if bid >= trailing ask - margin*spread then SELL 

    # if not falling
    # spread = abs(trailing ask - training bid)
    # if ask <= trailing bid + margin*spread then BUY 

    # inputs:
    # avg_w - averaging window size to get ask and bid trends
    # rise_th - a threshold to detect market rising situation
    # fall_th - a threshold to detect market rising situation
    # spread_trade_margin - a margin from the ask/bid trends at which mm trade is considered profitable 

    # Trends
    df['trailing_ask'] = df['ask'].rolling(avg_w).mean()
    df['trailing_bid'] = df['bid'].rolling(avg_w).mean()
    # Spread
    df['spread'] = (df['trailing_ask']-df['trailing_bid']).abs()
    # Price gradient trend
    df['gradient'] = (df['trailing_bid']+df['trailing_ask'])/(2*df['trailing_ask'])
    df['gradient'] = df['gradient'].diff()
    
    df['operation'] = df.apply(compute_trading_decision, axis = 1, rise_th = rise_th, fall_th = fall_th, spread_trade_margin = spread_trade_margin )



In [43]:
# Lets compute trades with above strategy

# Props (see description in the above function)
props = {'spread_trade_margin'  : 0.2,
         'rise_th'              : 0.05, 
         'fall_th'              : 0.05,
         'avg_w'                : 5}

# compute for assetA
dfA = pd.DataFrame()
dfA['ask'] = df['assetA_ask']
dfA['bid'] = df['assetA_bid']
compute_trades(dfA, **props)

# compute for assetB
dfB = pd.DataFrame()
dfB['ask'] = df['assetB_ask']
dfB['bid'] = df['assetB_bid']
compute_trades(dfB, **props)

# constructing action df
dfOperations = pd.DataFrame()
dfOperations['ts'] = df['timestamp']
dfOperations['actionA'] = dfA['operation']
dfOperations['actionB'] = dfB['operation']

# unify and write the operations to output.json
def produce_output_dict(df):
    output = []
    last_trade = 0
    df = df[df.actionB.isin(['sell', 'buy']) | df.actionA.isin(['sell', 'buy'])]
    for index, row in df.iterrows():
        actions = []
        if row['actionA'] == 'sell':
            actions.append('sellA')
        if row['actionA'] == 'buy':
            actions.append('buyA')
        if row['actionB'] == 'sell':
            actions.append('sellB')
        if row['actionB'] == 'buy':
            actions.append('buyB')

        ts = int(row['ts'])

        if len(actions)>0 and ts-last_trade>=30*1000:
            output.append({"time": int(row['ts']), "actions": actions})
            last_trade = ts

    return output

import json
output = produce_output_dict(dfOperations)
with open('output.json', 'w') as fp:
    json.dump(output, fp)


SyntaxError: invalid syntax (<ipython-input-43-fe2c1ab91653>, line 4)